In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
from torchvision.io import read_image
from torchvision.transforms import Resize

import torch as tn
import torch.nn as nn
import torchtt as tntt

from torch import optim
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
import numpy as np

from time import time

device = tn.device('cuda' if tn.cuda.is_available() else 'cpu')

/home/usr/myenv/lib/python3.12/site-packages/torchtt/_dmrg.py:19: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/home/usr/myenv/lib/python3.12/site-packages/torchtt/_amen.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/home/usr/myenv/lib/python3.12/site-packages/torchtt/solvers.py:21: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(
/home/usr/myenv/lib/python3.12/site-packages/torchtt/cpp.py:12: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn("\x1B\nC++ implementation not available. Using pure Python.\n\033")
/home/usr/myenv/lib/python3.12/site-packages/torchtt/__init__.py:34: UserWarning: 
C++ implementation not available. Using pure Python.

  warnings.warn(


In [3]:
path1 = "data/1_1.jpg"
path2 = "data/1_2.jpg"
path3 = "data/1_3.jpg"
res = Resize((256, 256))

im1 = res(read_image(path1)/255.0)
im2 = res(read_image(path2)/255.0)
im3 = res(read_image(path3)/255.0)

In [4]:
im1_tt = tntt.TT(im1)
im2_tt = tntt.TT(im2)
im3_tt = tntt.TT(im3)

In [5]:
test2 = tntt.kron(im1_tt, im2_tt)
test3 = tntt.kron(test2, im3_tt)

In [6]:
test3

TT with sizes and ranks:
N = [3, 256, 256, 3, 256, 256, 3, 256, 256]
R = [1, 3, 256, 1, 3, 256, 1, 3, 256, 1]

Device: cpu, dtype: torch.float32
#entries 786459 compression 1.034838387820653e-10

In [7]:
class BasicTT(nn.Module):
    def __init__(self):
        super().__init__()
        self.ttl1 = tntt.nn.LinearLayerTT([1, 500, 500, 500], 
                                          [8, 100, 100, 100],
                                          [1, 2  , 2  , 2   , 1])
        
        self.ttl2 = tntt.nn.LinearLayerTT([8, 100, 100, 100],
                                          [8, 50 , 50 , 50 ], 
                                          [1, 5  , 5  , 5   , 1])

        self.ttl3 = tntt.nn.LinearLayerTT([8, 50, 50, 50],
                                          [2, 8, 8, 8],
                                          [1,10,10, 10, 1])
        
        self.linear1 = nn.Linear(1024, 512, dtype = tn.float32)

        self.linear2 = nn.Linear(512, 10, dtype = tn.float32)
        self.logsoftmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.ttl1(x)
        x = tn.relu(x)
        x = self.ttl2(x)
        x = tn.relu(x)
        x = self.ttl3(x)
        x = tn.relu(x)
        
        x = x.view(-1,1024)
        x = self.linear1(x)
        x = tn.relu(x)
        x = self.linear2(x)
        return self.logsoftmax(x)

In [8]:
model = BasicTT().to(device)
loss_function = nn.CrossEntropyLoss()   
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [9]:
test_inp = tn.rand((1,500,500,500)).cuda()
target = tn.rand((1,10)).cuda()

optimizer.zero_grad()
output = model(test_inp)
loss = loss_function(output, target)
loss.backward()
optimizer.step()

In [10]:
# ОЗУ требует в 7.8 раз больше времени на шаг оптимизации

# ИЗВЛЕКАЕМ ПРИЗННАКИ ИЗОБРАЖЕНИЙ

In [13]:
from torchvision.models import resnet50, ResNet50_Weights, vgg16_bn, VGG16_BN_Weights
from torchsummary import summary

DETECTION_CLASSES = 6

class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        

        self.conv = resnet50(weights=ResNet50_Weights.DEFAULT)
        for param in self.conv.parameters():
            param.requires_grad = False

        self.fc1 = nn.Sequential(
            nn.Linear(in_features = 1000, out_features = 500, bias=True),
            nn.BatchNorm1d(500, momentum = 0.5),
            nn.LeakyReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 500, out_features = 500, bias=True),
            nn.BatchNorm1d(500, momentum = 0.5),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 500, out_features = 500, bias=True),
            nn.BatchNorm1d(500, momentum = 0.5),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features = 500, out_features = DETECTION_CLASSES, bias=True),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 1000)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [38]:
from glob import glob
class ImageTriplets(Dataset):
    def __init__(self, data_path):
        self.classes = []
        self.all_locations = {}
        self.total_images = 0
        
        for cls in glob(f"{data_path}/*"):
            cls_clear = cls.split('/')[-1]
            self.classes.append(cls_clear)
            
            self.all_locations[cls_clear] = []
            for obj_path in glob(f"{cls}/*"):
                obj = obj_path.split('/')[-1]
                images = glob(f"{obj_path}/images/*")
                
                self.all_locations[cls_clear].append(images)
                self.total_images += len(images)
            

        self.classes.sort()
        
    def __getitem__(self, index):
        pass

In [39]:
data = ImageTriplets("/home/usr/Desktop/co3d")

In [40]:
data.all_locations.keys()

dict_keys(['bench', 'plant', 'book', 'umbrella', 'motorcycle', 'laptop'])

In [41]:
data.all_locations['bench'][0]

['/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000100.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000053.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000095.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000024.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000002.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000114.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000065.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000124.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000090.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000049.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000012.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000055.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame000105.jpg',
 '/home/usr/Desktop/co3d/bench/349_36542_68793/images/frame00004